In [108]:
import numpy as np
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score 
import warnings
warnings.filterwarnings("ignore")
random_state = 123456789
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
import matplotlib.pylab as plt
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
import matplotlib.pylab as plt

In [109]:
df = pd.read_csv('preprocessed_data.csv')
label = 'price'
y = df[label]
columns = ['price']
df.drop(columns, inplace=True, axis=1)
feature_namelist = []
for item in df.columns:
    feature_namelist.append(item)
#print(len(feature_namelist))
feature_names = feature_namelist
label = 'price'
X = df[feature_names]

In [110]:
# feature_names = feature_namelist
# label = 'price'
# df = pd.read_csv('preprocessed_data.csv')
# df.columns = feature_names+[label]
# df.replace(to_replace='?', value=np.nan, inplace=True)
# df=df.astype(float)
# df=df.dropna()
# df.reset_index(drop=True, inplace=True)
# X=df[feature_names]
# y=df[label]
# classes, counts = np.unique(y,return_counts=True)
# print('balance:',np.max(counts/len(y)))

In [111]:
df = pd.read_csv("preprocessed_for_grid.csv")
numeric_features_minmax = ['latitude', 'longitude', 'availability_365']
numeric_features_standard = ['minimum_nights','number_of_reviews','reviews_per_month',
                             'calculated_host_listings_count', 'time']
categorical_features = ['neighbourhood_group','neighbourhood', 'room_type','race', 'name']
label = 'price'

In [123]:
X=df[numeric_features_minmax+numeric_features_standard+categorical_features]
y=df[label]
classes, counts = np.unique(y,return_counts=True)
#print(counts/len(y))
print('balance:',np.max(counts/len(y)))

balance: 0.04194702934860415


In [120]:
n_folds = 5
random_state = 2000
def ML_pipeline_kfold_GridSearchCV(X, y, random_state, n_folds, model, param_grid):
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)
    
    numeric_features_minmax = ['availability_365','latitude','longitude']
    numeric_features_standard = ['minimum_nights','number_of_reviews','reviews_per_month',
                                 'calculated_host_listings_count','time'] 
    
    numeric_transformer_minmax = Pipeline(steps=[('minmaxscaler', MinMaxScaler())])
    numeric_transformer_standard = Pipeline(steps=[('standardscaler', StandardScaler())])
    
    categorical_features = ['neighbourhood_group','neighbourhood','room_type']
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder())])
    
    preprocessor = ColumnTransformer(remainder='passthrough',transformers=[
        ('num_minmax', numeric_transformer_minmax, numeric_features_minmax),
        ('num_standard', numeric_transformer_standard, numeric_features_standard)
        ('cat', categorical_transformer, categorical_features)])
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor),('classifier',model)])
    
    grid = GridSearchCV(pipe, param_grid=param_grid, scoring=make_scorer(accuracy_score), cv=kf, 
                        return_train_score=True, n_jobs=-1) 
    
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test) 
    

In [117]:
model = LogisticRegression(penalty='l1', solver = 'saga', max_iter=10000, multi_class='auto')
param_grid = {'classifier__C': 1/np.logspace(-4,4,20)}
best_model, best_score = ML_pipeline_kfold_GridSearchCV(X, y, random_state, n_folds, model, param_grid)
c = best_model.best_params_['classifier__C']
print(best_score)
print(c)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.